# Funciones para evaluar los supuestos de la Regresion Lineal

In [1]:
# librerias
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [1]:
# Media de los residuales
def Med_residual(model_entrenado):
    '''
    Calcula la media de los residuos del modelo. Solicita como parametro
    el modelo entrenado
    '''
    resid_val = model_entrenado.resid
    print(resid_val.mean())
    
# Normalidad
def Norm_residuales(model_entrenado, X_test, y_test):
    '''
    Muestra la grafica Q-Q de los residuales asi como los resultados de la
    prueba Shapiro Wilk. Requiere como parametro el modelo entrenado
    '''
    #resid_val = model_entrenado.resid
    y_pred = model_entrenado.predict(X_test)
    residuales = y_test - y_pred
    
    sm.qqplot(residuales, line='s')
    plt.title('Grafico Q-Q', fontsize=16);
    
    print(stats.shapiro(residuales))
    
# Linealidad
def Linealidad(model_entrenado, X_train, y_train):
    '''
    Grafica los valores observados con los valores predichos por el modelo. Requiere
    como parametro el modelo entrenado y los datos con los que fue entrenado el modelo (y_train)
    '''
    predict_val = model_entrenado.predict(X_train)
    
    fig, ax = plt.subplots(figsize=(7,4))
    sns.regplot(x=predict_val, y=y_train, lowess=True, line_kws={'color':'red'})
    ax.set_title('Valores predichos vs observados', fontsize=16);
    ax.set_xlabel('Predicciones', fontsize=13)
    ax.set_ylabel('Valores Reales', fontsize=13);
    
# Homocedasticidad
def Homocedasticidad(model_entrenado):
    '''
    Evalua la igualdad de varianza de los residuales mostrando una grafica de dispersion
    entre los valores predichos y los residuales. Ademas, muestra el p-value obtenido de
    la prueba Breush - Pagan para contrastar la evaluacion. Requiere como parametro el
    modelo entrenado
    '''
    # valores residuales estandarizados
    resid_stand = model_entrenado.get_influence().resid_studentized_internal
    # valores de la prediccion del modelo
    predict_val = model_entrenado.predict()
    # grafico
    sns.regplot(x=predict_val, y=resid_stand, lowess=True, line_kws={'color':'red'})
    plt.title('Valores predichos vs Residuales');
    
    # prueba de Breush-Pragan
    resid_val = model_entrenado.resid
    BP_test = sms.stats.diagnostic.het_breuschpagan(resid_val, model_entrenado.model.exog)
    print('\nBreush - Pagan Test: p-value=', BP_test[1])
    
# Multicolinealidad
def Multicolinealidad(X):
    '''
    Examina la multicolinealidad de las variables con la metrica VIF.
    Requiere como parametro los datos de las variables predictoras
    '''
    # Columna para el intercepto
    X_constant = sm.add_constant(X)
    # calculo de vif
    vif = [variance_inflation_factor(X_constant.values, i) for i in range(X_constant.shape[1])]
    return pd.DataFrame({'vif': vif[1:]}, index=X.columns).T